## 라이브러리

In [372]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import f1_score

import requests
import json
from datetime import datetime
import calendar
from dateutil.relativedelta import relativedelta

In [373]:
import koreanize_matplotlib

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

## 데이터

In [374]:
train = pd.read_csv('train.csv').copy()
test = pd.read_csv('test.csv').copy()
submission = pd.read_csv('sample_submission.csv')

train.shape, test.shape, submission.shape

((1205, 12), (50, 10), (50, 3))

In [375]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [376]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


## 전처리

### 데이터 타입 변경

In [377]:
# 일자, 현본사소속재택근무자수, 중식계, 석식계 데이터 타입 변경

# train
train["일자"] = pd.to_datetime(train["일자"])
train["현본사소속재택근무자수"] = train["현본사소속재택근무자수"].astype("int")
train["중식계"] = train["중식계"].astype("int")
train["석식계"] = train["석식계"].astype("int")
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   일자              1205 non-null   datetime64[ns]
 1   요일              1205 non-null   object        
 2   본사정원수           1205 non-null   int64         
 3   본사휴가자수          1205 non-null   int64         
 4   본사출장자수          1205 non-null   int64         
 5   본사시간외근무명령서승인건수  1205 non-null   int64         
 6   현본사소속재택근무자수     1205 non-null   int32         
 7   조식메뉴            1205 non-null   object        
 8   중식메뉴            1205 non-null   object        
 9   석식메뉴            1205 non-null   object        
 10  중식계             1205 non-null   int32         
 11  석식계             1205 non-null   int32         
dtypes: datetime64[ns](1), int32(3), int64(4), object(4)
memory usage: 99.0+ KB


In [378]:
# test
test["일자"] = pd.to_datetime(test["일자"])
test["현본사소속재택근무자수"] = test["현본사소속재택근무자수"].astype("int")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   일자              50 non-null     datetime64[ns]
 1   요일              50 non-null     object        
 2   본사정원수           50 non-null     int64         
 3   본사휴가자수          50 non-null     int64         
 4   본사출장자수          50 non-null     int64         
 5   본사시간외근무명령서승인건수  50 non-null     int64         
 6   현본사소속재택근무자수     50 non-null     int32         
 7   조식메뉴            50 non-null     object        
 8   중식메뉴            50 non-null     object        
 9   석식메뉴            50 non-null     object        
dtypes: datetime64[ns](1), int32(1), int64(4), object(4)
memory usage: 3.8+ KB


### 파생변수 만들기
연도, 월, 일, 출근인원, 중식_신메뉴, 석식_신메뉴, 공휴일, 조리법, 식재료

In [379]:
# train
train["연도"] = train["일자"].dt.year
train["월"] = train["일자"].dt.month
train["일"] = train["일자"].dt.day
train["요일"] = train['일자'].dt.weekday + 1

train.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,연도,월,일
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039,331,2016,2,1
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867,560,2016,2,2


In [380]:
# test
test["연도"] = test["일자"].dt.year
test["월"] = test["일자"].dt.month
test["일"] = test["일자"].dt.day
test["요일"] = test['일자'].dt.weekday + 1

test.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,연도,월,일
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2021,1,27
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2021,1,28


In [381]:
# 회사에 출근한 사람 : 본사정원수 - (본사휴가자수 + 본사출장자수 + 현본사소속재택근무자수)
train["출근인원"] = train["본사정원수"] - (train["본사휴가자수"] + train["본사출장자수"] + train["현본사소속재택근무자수"])
train.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,연도,월,일,출근인원
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039,331,2016,2,1,2401
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867,560,2016,2,2,2378


In [382]:
# 회사에 출근한 사람 : 본사정원수 - (본사휴가자수 + 본사출장자수 + 현본사소속재택근무자수)
test["출근인원"] = test["본사정원수"] - (test["본사휴가자수"] + test["본사출장자수"] + test["현본사소속재택근무자수"])
test.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,연도,월,일,출근인원
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2021,1,27,2355
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2021,1,28,2319


In [383]:
# 중식메뉴, 석식메뉴에 'New'가 포함되어 있으면 1, 없으면 0으로 이루어진 '신메뉴' 컬럼 생성
train['중식_신메뉴'] = (train['중식메뉴'].str.contains('New')).astype(int)
train['석식_신메뉴'] = (train['석식메뉴'].str.contains('New')).astype(int)
train.head(1)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,연도,월,일,출근인원,중식_신메뉴,석식_신메뉴
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039,331,2016,2,1,2401,0,0


In [384]:
# test
test['중식_신메뉴'] = (test['중식메뉴'].str.contains('New')).astype(int)
test['석식_신메뉴'] = (test['석식메뉴'].str.contains('New')).astype(int)
test.head(1)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,연도,월,일,출근인원,중식_신메뉴,석식_신메뉴
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2021,1,27,2355,0,1


In [385]:
# 공휴일 

In [386]:
# url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo"

# AUTH_KEY = "###################################"


# params = {
#     'solYear':str(2020),
#     'solMonth':str(1).zfill(2),
#     '_type':'json',
#     'ServiceKey' : AUTH_KEY
# }

# response =  requests.get(url,params=params)
# print(response.text)

# result = json.loads(response.text)

In [387]:
# def getHolidays(year,month,key):
#     url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
#     AUTH_KEY = "###################################"
        
#     params = {
#         'solYear':str(year),
#         'solMonth':str(month).zfill(2),
#         '_type':'json',
#         'ServiceKey' : AUTH_KEY
#     }

#     res = requests.get(url,params=params)
#     dic = json.loads(res.text)
#     counts = dic['response']['body']['totalCount']

#     if counts < 1 :
#         return []

#     item =  dic['response']['body']['items']['item']

#     if counts == 1:
#         return [item]

#     return item

In [388]:
# holidays =[]
# AUTH_KEY = "###################################"
# for year in range(2016,2022):
#     for month in range(1,13):
#         holidays.extend(getHolidays(year,month,AUTH_KEY))

In [389]:
# df_holiday = pd.DataFrame(holidays,columns=['locdate','dateName'])
# df_holiday.columns = ["일자", "공휴일"] 
# df_holiday.to_csv('./holiday.csv',index=None)
# pd.read_csv("./holiday.csv")

In [390]:
# 휴일 전날
holi = pd.read_csv("./holiday.csv")
holi["일자"] = holi["일자"].astype(str)
holiday_before_df = holi.copy()
holiday_before_df['일자'] = holiday_before_df['일자'].apply(lambda x : datetime.strptime(x,'%Y%m%d'))
holiday_before_df['일자'] = holiday_before_df['일자'].apply(lambda x : x - relativedelta(days=1))
holiday_before_df['일자'] = holiday_before_df['일자'].apply(lambda x : datetime.strftime(x, '%Y-%m-%d'))
holiday_before_df['공휴일'] = holi['공휴일'].apply(lambda x : 1)
holiday_before_df.columns = ['일자', '휴일전날']

holiday_before_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      107 non-null    object
 1   휴일전날    107 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [391]:
holiday_before_df.head(2)

,일자,휴일전날
0,2015-12-31,1
1,2016-02-06,1


In [392]:
# 휴일전날 merge
train["일자"] = train["일자"].astype(str)
train = pd.merge(train, holiday_before_df, left_on='일자', right_on='일자', how='left')
before_friday = train[train['요일']==5].index
train['휴일전날'][before_friday] = 1

train.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_4816\3845009856.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['휴일전날'][before_friday] = 1


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,연도,월,일,출근인원,중식_신메뉴,석식_신메뉴,휴일전날
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039,331,2016,2,1,2401,0,0,NaN
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867,560,2016,2,2,2378,0,0,NaN


In [393]:
# 평일은 0 처리
train['휴일전날'] = train['휴일전날'].fillna(0)

In [394]:
train['휴일전날'].isnull().sum()

0

In [395]:
train[train['휴일전날'] == 1].head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,연도,월,일,출근인원,중식_신메뉴,석식_신메뉴,휴일전날
4,2016-02-05,5,2601,278,181,34,0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925,330,2016,2,5,2142,0,0,1.0
6,2016-02-12,5,2601,389,156,93,0,모닝롤/야채샌드 우유/두유/주스 계란후라이 고구마죽/쌀밥 (쌀:국내산) 봄동된...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 꽃게탕 돈육굴소스볶음 옥수수전 유채나물 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치콩나물국 미니함박 어묵볶음 물파래무침 ...",909,598,2016,2,12,2056,0,0,1.0


In [396]:
# 휴일전날 merge
test["일자"] = test["일자"].astype(str)
test = pd.merge(test, holiday_before_df, left_on='일자', right_on='일자', how='left')
before_friday = test[test['요일']==5].index
test['휴일전날'][before_friday] = 1

test.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_4816\936886698.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['휴일전날'][before_friday] = 1


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,연도,월,일,출근인원,중식_신메뉴,석식_신메뉴,휴일전날
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2021,1,27,2355,0,1,NaN
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2021,1,28,2319,0,0,NaN


In [397]:
# 평일은 0 처리
test['휴일전날'] = test['휴일전날'].fillna(0)

In [398]:
test['휴일전날'].isnull().sum()

0

In [399]:
test[test['휴일전날'] == 1].head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,연도,월,일,출근인원,중식_신메뉴,석식_신메뉴,휴일전날
2,2021-01-29,5,2983,270,249,0,294,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,2021,1,29,2170,0,0,1.0
7,2021-02-05,5,2924,169,252,0,256,모닝롤/앙버터모닝빵 우유/주스 계란후라이/찐계란 고구마죽/흑미밥 옹심이국 머위나물무...,쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러...,유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치,2021,2,5,2247,0,0,1.0


In [400]:
# 조리법

In [401]:
train['중식메뉴_1'] = train['중식메뉴'].str.split(' ')
train['석식메뉴_1'] = train['석식메뉴'].str.split(' ')

test['중식메뉴_1'] = test['중식메뉴'].str.split(' ')
test['석식메뉴_1'] = test['석식메뉴'].str.split(' ')

In [402]:
def get_token(data) :
    tokens = []
    for token in data :#하루의 중식 메뉴 
        s_list = []
        for t in token : #중식 메뉴 중 밥, 국,...
            if t.startswith('(') :
                s_list.append(t)
            elif (t.startswith('(') == False) & (len(t) > 1) :
                s_list.append(t)
            else :
                pass
        tokens.append(s_list)
    return tokens

In [403]:
train['중식_토큰'] = get_token(train['중식메뉴_1'])
train['석식_토큰'] = get_token(train['석식메뉴_1'])

In [404]:
test['중식_토큰'] = get_token(test['중식메뉴_1'])
test['석식_토큰'] = get_token(test['석식메뉴_1'])

In [405]:
def get_ingredient(data) :
    
    ing_df = pd.DataFrame(np.zeros((data.shape[0], 7)), columns = ['해산물', '소', '돼지', '닭', '오리', '채소', '재료_기타'])

    for t in range(data.shape[0]):
        token = data.중식_토큰.str[2][t]
        if '연어' in token or'골뱅이' in token or'열기' in token or'조기' in token or'탕수어' in token or'양장피' in token or'홍어' in token or'명태' in token or'적어' in token or'장어' in token or'동태' in token or'산슬' in token or'코다리' in token or'가자미' in token or'해물' in token or'생선' in token or'새우' in token or'꽁치' in token or'갈치' in token or'임연수' in token or'삼치' in token or'고등어' in token or'굴비' in token or'오징어' in token or'쭈꾸미' in token or'주꾸미' in token or'낙지' in token or'문어' in token :
            ing_df.head().at[t, '해산물'] = 1
        elif '왕갈비' in token or'소갈비' in token or'장조림' in token or'불고기' in token or'차돌' in token or'육전' in token or'너비아니' in token or'떡갈비' in token or(token.startswith('소') & (token.startswith('소세') == False)) or '함박' in token or'쇠고기' in token or'소고기' in token or'쇠' in token :
            ing_df.at[t, '소'] = 1
        elif '궁보계정' in token or'삼계탕' in token or'윙' in token or'유린기' in token or'깐풍'in token or'닭' in token or'치킨' in token or'후라이드' in token :
            ing_df.at[t, '돼지'] = 1
        elif '폭립' in token or'오향장육' in token or'동파육' in token or'히레카츠' in token or'순대' in token or'미트볼' in token or'등갈비' in token or'소세지' in token or'목살' in token or'탕수육' in token or'제육' in token or'돈' in token or'돼지' in token or'두루치기' in token or'삼겹' in token or'보쌈' in token or'족발' in token :
            ing_df.at[t, '닭'] = 1
        elif '오리' in token :
            ing_df.at[t, '오리'] = 1
        elif token.endswith('두부') or '꼬치산적' in token or '고추' in token or'양파' in token or'부추' in token or'고구마' in token or'감자' in token or'깻잎' in token or'샐러드' in token or'시금치' in token or'야채' in token :
            ing_df.at[t, '채소'] = 1
        else :
            ing_df.at[t, '재료_기타'] = 1
            pass
    return ing_df

In [406]:
train = pd.concat([train, get_ingredient(train)], axis = 1)
test = pd.concat([test, get_ingredient(test)], axis = 1)

C:\Users\user\AppData\Local\Temp\ipykernel_4816\46408424.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ing_df.head().at[t, '해산물'] = 1
C:\Users\user\AppData\Local\Temp\ipykernel_4816\46408424.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ing_df.head().at[t, '해산물'] = 1
C:\Users\user\AppData\Local\Temp\ipykernel_4816\46408424.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ing_df.head().at[t, '해산물'] = 1
C:\Users\user\AppDa

In [407]:
train['중식_메인요리'] = train.중식_토큰.str[3]
test['중식_메인요리'] = test.중식_토큰.str[2]

In [408]:
train['석식_메인요리'] = 0
if train.shape[0] < 1067:
  train['석식_메인요리'] = train.석식_토큰.str[3]
else:
  train['석식_메인요리'] = train.석식_토큰.str[2]
test['석식_메인요리'] = test.석식_토큰.str[2]

In [409]:
#조리법 종류 별 컬럼 생성 함수
def get_recipe(data, col) :
    tm = col[:2]
    cat = ['전', '무침','튀김', '찜', '볶음', '조림', '구이', '훈제', '조리_기타']
    recipe_df = pd.DataFrame(np.zeros((data.shape[0], 9)), columns = [f'{tm}_{x}' for x in cat])

    for t in range(data.shape[0]) :
        try :
            token = data[col][t]
            if '고추잡채' in token or '궁보계정' in token or '산슬' in token or token.endswith('잡채') or '마파두부' in token or '두루치기' in token or '닭갈비' in token or token.endswith('볶음') or '볶음' in token :
                recipe_df.at[t, f'{tm}_볶음'] = 1 
            elif token.endswith('데리야끼') or token.endswith('립') or '함박' in token or '그라탕' in token or token.endswith('갈비') or '주물럭' in token or '스테이크' in token or token.endswith('구이') or '불고기' in token or '구이' in token :
                recipe_df.at[t, f'{tm}_구이'] = 1
            elif '전병' in token or token.endswith('전') :
                recipe_df.at[t, f'{tm}_전'] = 1
            elif token.endswith('김치말이') or token.endswith('만두') or '보쌈' in token or '수육' in token or token.endswith('찜') or '찜' in token :
                recipe_df.at[t, f'{tm}_찜'] = 1
            elif '파채' in token or token.endswith('무침') or token.endswith('샐러드') or '양장피' in token :
                recipe_df.at[t, f'{tm}_무침'] = 1
            elif '오향장육' in token or '동파육' in token or token.endswith('조림') :
                recipe_df.at[t, f'{tm}_조림'] = 1
            elif '튀김' in token or '통닭' in token or token.endswith('새우') or '강정' in token or '미트볼' in token or '프리타타' in token or '카츠' in token or '깐풍' in token or '고로케' in token or '유린기' in token or '탕수' in token or token.endswith('닭') or token.endswith('치킨') or token.endswith('튀김') or '너겟' in token or token.endswith('강정') or '가스' in token or '까스' in token or '핑거' in token or '텐더' in token or '커틀렛' in token or '커틀릿' in token :
                recipe_df.at[t, f'{tm}_튀김'] = 1
            elif '훈제' in token :
                recipe_df.at[t, f'{tm}_훈제'] = 1
            else :
                recipe_df.at[t, f'{tm}_조리_기타'] = 1
        except :
            recipe_df.at[t, f'{tm}_조리_기타'] = 1
    return recipe_df

In [410]:
train = pd.concat([train, get_recipe(train, '중식_메인요리')], axis = 1)
train = pd.concat([train, get_recipe(train, '석식_메인요리')], axis = 1)

In [411]:
train['중식메인조리법'] = 0
for t in range(train.shape[0]) :
  
  token = train['중식_메인요리'][t]
  if '고추잡채' in token or '궁보계정' in token or '산슬' in token or token.endswith('잡채') or '마파두부' in token or '두루치기' in token or '닭갈비' in token or token.endswith('볶음') or '볶음' in token :
      train.loc[t, '중식메인조리법'] = 1 
  elif token.endswith('데리야끼') or token.endswith('립') or '함박' in token or '그라탕' in token or token.endswith('갈비') or '주물럭' in token or '스테이크' in token or token.endswith('구이') or '불고기' in token or '구이' in token :
      train.loc[t, '중식메인조리법'] = 2
  elif '전병' in token or token.endswith('전') :
      train.loc[t, '중식메인조리법'] = 3
  elif token.endswith('김치말이') or token.endswith('만두') or '보쌈' in token or '수육' in token or token.endswith('찜') or '찜' in token :
      train.loc[t, '중식메인조리법'] = 4
  elif '파채' in token or token.endswith('무침') or token.endswith('샐러드') or '양장피' in token :
      train.loc[t, '중식메인조리법'] = 5
  elif '오향장육' in token or '동파육' in token or token.endswith('조림') :
      train.loc[t, '중식메인조리법'] = 6
  elif '통닭' in token or '튀김' in token or token.endswith('새우') or '강정' in token or '미트볼' in token or '프리타타' in token or '카츠' in token or '깐풍' in token or '고로케' in token or '유린기' in token or '탕수' in token or token.endswith('닭') or token.endswith('치킨') or token.endswith('튀김') or '너겟' in token or token.endswith('강정') or '가스' in token or '까스' in token or '핑거' in token or '텐더' in token or '커틀렛' in token or '커틀릿' in token :
      train.loc[t, '중식메인조리법'] = 7
  elif '훈제' in token :
      train.loc[t, '중식메인조리법'] = 8
  else :
      train.loc[t, '중식메인조리법'] = 9

In [412]:
train['석식메인조리법'] = 0
for t in range(train.shape[0]) :
  
  token = str(train['석식_메인요리'][t])
  if '고추잡채' in token or '궁보계정' in token or '산슬' in token or token.endswith('잡채') or '마파두부' in token or '두루치기' in token or '닭갈비' in token or token.endswith('볶음') or '볶음' in token :
      train.loc[t, '석식메인조리법'] = 1 
  elif token.endswith('데리야끼') or token.endswith('립') or '함박' in token or '그라탕' in token or token.endswith('갈비') or '주물럭' in token or '스테이크' in token or token.endswith('구이') or '불고기' in token or '구이' in token :
      train.loc[t, '석식메인조리법'] = 2
  elif '전병' in token or token.endswith('전') :
      train.loc[t, '석식메인조리법'] = 3
  elif token.endswith('김치말이') or token.endswith('만두') or '보쌈' in token or '수육' in token or token.endswith('찜') or '찜' in token :
      train.loc[t, '석식메인조리법'] = 4
  elif '파채' in token or token.endswith('무침') or token.endswith('샐러드') or '양장피' in token :
      train.loc[t, '석식메인조리법'] = 5
  elif '오향장육' in token or '동파육' in token or token.endswith('조림') :
      train.loc[t, '석식메인조리법'] = 6
  elif '통닭' in token or '튀김' in token or token.endswith('새우') or '강정' in token or '미트볼' in token or '프리타타' in token or '카츠' in token or '깐풍' in token or '고로케' in token or '유린기' in token or '탕수' in token or token.endswith('닭') or token.endswith('치킨') or token.endswith('튀김') or '너겟' in token or token.endswith('강정') or '가스' in token or '까스' in token or '핑거' in token or '텐더' in token or '커틀렛' in token or '커틀릿' in token :
      train.loc[t, '석식메인조리법'] = 7
  elif '훈제' in token :
      train.loc[t, '석식메인조리법'] = 8
  else :
      train.loc[t, '석식메인조리법'] = 9

In [413]:
test = pd.concat([test, get_recipe(test, '중식_메인요리')], axis = 1)

In [414]:
test = pd.concat([test, get_recipe(test, '석식_메인요리')], axis = 1)

In [415]:
test['중식메인조리법'] = 0
for t in range(test.shape[0]) :
  
  token = test['중식_메인요리'][t]
  if '고추잡채' in token or '궁보계정' in token or '산슬' in token or token.endswith('잡채') or '마파두부' in token or '두루치기' in token or '닭갈비' in token or token.endswith('볶음') or '볶음' in token :
      test.loc[t, '중식메인조리법'] = 1 
  elif token.endswith('데리야끼') or token.endswith('립') or '함박' in token or '그라탕' in token or token.endswith('갈비') or '주물럭' in token or '스테이크' in token or token.endswith('구이') or '불고기' in token or '구이' in token :
      test.loc[t, '중식메인조리법'] = 2
  elif '전병' in token or token.endswith('전') :
      test.loc[t, '중식메인조리법'] = 3
  elif token.endswith('김치말이') or token.endswith('만두') or '보쌈' in token or '수육' in token or token.endswith('찜') or '찜' in token :
      test.loc[t, '중식메인조리법'] = 4
  elif '파채' in token or token.endswith('무침') or token.endswith('샐러드') or '양장피' in token :
      test.loc[t, '중식메인조리법'] = 5
  elif '오향장육' in token or '동파육' in token or token.endswith('조림') :
      test.loc[t, '중식메인조리법'] = 6
  elif '통닭' in token or '튀김' in token or token.endswith('새우') or '강정' in token or '미트볼' in token or '프리타타' in token or '카츠' in token or '깐풍' in token or '고로케' in token or '유린기' in token or '탕수' in token or token.endswith('닭') or token.endswith('치킨') or token.endswith('튀김') or '너겟' in token or token.endswith('강정') or '가스' in token or '까스' in token or '핑거' in token or '텐더' in token or '커틀렛' in token or '커틀릿' in token :
      test.loc[t, '중식메인조리법'] = 7
  elif '훈제' in token :
      test.loc[t, '중식메인조리법'] = 8
  else :
      test.loc[t, '중식메인조리법'] = 9

In [416]:
test['석식메인조리법'] = 0
for t in range(test.shape[0]) :
  
  token = str(test['석식_메인요리'][t])
  if '고추잡채' in token or '궁보계정' in token or '산슬' in token or token.endswith('잡채') or '마파두부' in token or '두루치기' in token or '닭갈비' in token or token.endswith('볶음') or '볶음' in token :
      test.loc[t, '석식메인조리법'] = 1 
  elif token.endswith('데리야끼') or token.endswith('립') or '함박' in token or '그라탕' in token or token.endswith('갈비') or '주물럭' in token or '스테이크' in token or token.endswith('구이') or '불고기' in token or '구이' in token :
      test.loc[t, '석식메인조리법'] = 2
  elif '전병' in token or token.endswith('전') :
      test.loc[t, '석식메인조리법'] = 3
  elif token.endswith('김치말이') or token.endswith('만두') or '보쌈' in token or '수육' in token or token.endswith('찜') or '찜' in token :
      test.loc[t, '석식메인조리법'] = 4
  elif '파채' in token or token.endswith('무침') or token.endswith('샐러드') or '양장피' in token :
      test.loc[t, '석식메인조리법'] = 5
  elif '오향장육' in token or '동파육' in token or token.endswith('조림') :
      test.loc[t, '석식메인조리법'] = 6
  elif '통닭' in token or '튀김' in token or token.endswith('새우') or '강정' in token or '미트볼' in token or '프리타타' in token or '카츠' in token or '깐풍' in token or '고로케' in token or '유린기' in token or '탕수' in token or token.endswith('닭') or token.endswith('치킨') or token.endswith('튀김') or '너겟' in token or token.endswith('강정') or '가스' in token or '까스' in token or '핑거' in token or '텐더' in token or '커틀렛' in token or '커틀릿' in token :
      test.loc[t, '석식메인조리법'] = 7
  elif '훈제' in token :
      test.loc[t, '석식메인조리법'] = 8
  else :
      test.loc[t, '석식메인조리법'] = 9

In [417]:
train = train.drop(columns = ['중식메뉴_1', '석식메뉴_1',
       '중식_토큰', '석식_토큰', '해산물', '소', '돼지', '닭', '오리', '채소', '재료_기타', '중식_메인요리',
       '석식_메인요리'], axis = 1)
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '연도', '월', '일',
       '출근인원', '중식_신메뉴', '석식_신메뉴', '휴일전날', '중식_전', '중식_무침', '중식_튀김', '중식_찜',
       '중식_볶음', '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타', '석식_전', '석식_무침',
       '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타',
       '중식메인조리법', '석식메인조리법'],
      dtype='object')

In [418]:
test = test.drop(columns = ['중식메뉴_1',
       '석식메뉴_1', '중식_토큰', '석식_토큰', '해산물', '소', '돼지', '닭', '오리', '채소', '재료_기타',
       '중식_메인요리', '석식_메인요리'], axis = 1)
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '연도', '월', '일', '출근인원', '중식_신메뉴',
       '석식_신메뉴', '휴일전날', '중식_전', '중식_무침', '중식_튀김', '중식_찜', '중식_볶음', '중식_조림',
       '중식_구이', '중식_훈제', '중식_조리_기타', '석식_전', '석식_무침', '석식_튀김', '석식_찜', '석식_볶음',
       '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타', '중식메인조리법', '석식메인조리법'],
      dtype='object')

In [419]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식_무침,석식_튀김,석식_찜,석식_볶음,석식_조림,석식_구이,석식_훈제,석식_조리_기타,중식메인조리법,석식메인조리법
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,9
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,9
2,2016-02-03,3,2601,56,180,111,0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,9
3,2016-02-04,4,2601,104,220,355,0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,9
4,2016-02-05,5,2601,278,181,34,0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,9


In [420]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식_무침,석식_튀김,석식_찜,석식_볶음,석식_조림,석식_구이,석식_훈제,석식_조리_기타,중식메인조리법,석식메인조리법
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,9
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,9
2,2016-02-03,3,2601,56,180,111,0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,9
3,2016-02-04,4,2601,104,220,355,0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,9
4,2016-02-05,5,2601,278,181,34,0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,9


In [421]:
# 식재료

In [422]:
# train ln
lunch = []
for day in range(len(train)):
    tmp = train.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch.append(tmp)

In [423]:
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []

for i, day_menu in enumerate(lunch):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)
        
train_ln = train.copy()
train_ln['bob'] = bob
train_ln['gook'] = gook
train_ln['banchan1'] = banchan1
train_ln['banchan2'] = banchan2
train_ln['banchan3'] = banchan3
train_ln['kimchi'] = kimchi 
train_ln['side'] = side
train_ln.iloc[1066:1070, 7:]
# train_ln.info()

,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,연도,월,일,출근인원,중식_신메뉴,...,석식_조리_기타,중식메인조리법,석식메인조리법,bob,gook,banchan1,banchan2,banchan3,kimchi,side
1066,모닝롤/와플 우유/주스 계란후라이 해물죽/쌀밥 순두부탕 노각무침 포기김치 (...,"돈육버섯고추장덮밥 (쌀,돈육:국내산) 팽이무국 양파링카레튀김 모듬어묵볶음 참나...","쌀밥/잡곡밥 (쌀,현미,흑미:국내산) 부대찌개 (양지:호주산) 떡갈비조림 쫄면무침...",516,266,2020,6,12,2274,0,...,1.0,7,9,돈육버섯고추장덮밥,팽이무국,양파링카레튀김,모듬어묵볶음,참나물생채,포기김치,요구르트
1067,모닝롤/야채샌드 우유/주스 계란후라이 녹두죽/쌀밥 건새우무채국 쑥갓두부무침 오이보트...,쌀밥/잡곡밥 냉모밀국수 매운돈갈비찜 메밀전병*간장 고구마순볶음 포기김치 양상추샐러드...,쌀밥/잡곡밥 돈육김치찌개 깐풍기 고갈비구이 가지된장무침 포기김치,866,343,2020,7,1,2521,0,...,0.0,3,7,쌀밥/잡곡밥,냉모밀국수,매운돈갈비찜,메밀전병*간장,고구마순볶음,포기김치,양상추샐러드*딸기요거트
1068,모닝롤/크래미샌드 우유/주스 계란후라이 누룽지탕/쌀밥 참치김치찌개 느타리버섯장조림 ...,쌀밥/잡곡밥 대파육개장 홍어미나리초무침 어묵잡채 콩자반 배추겉절이 양상추샐러드*오리엔탈,"김밥 미니우동 구슬떡볶이 김말이,고추튀김 다시마채무초무침 포기김치",675,441,2020,7,2,2495,0,...,1.0,1,9,쌀밥/잡곡밥,대파육개장,홍어미나리초무침,어묵잡채,콩자반,배추겉절이,양상추샐러드*오리엔탈
1069,모닝롤/호떡 우유/주스 계란후라이 콩죽/쌀밥 북어무우국 베이컨숙주굴소스볶음 양상추샐...,카레라이스 동태알탕 부추고추전*간장 쫄면야채무침 과일요거트샐러드 포기김치 요구르트,쌀밥/잡곡밥 김치어묵국 케이준샐러드 섭산적표고굴소스볶음 매운콩나물무침 포기김치,607,269,2020,7,3,2291,0,...,0.0,5,5,카레라이스,동태알탕,부추고추전*간장,쫄면야채무침,과일요거트샐러드,포기김치,요구르트


In [424]:
# test ln 
lunch = []
for day in range(len(test)):
    tmp = test.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch.append(tmp)

In [425]:
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []

for i, day_menu in enumerate(lunch):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)

test_ln = test.copy()
test_ln['bob'] = bob
test_ln['gook'] = gook
test_ln['banchan1'] = banchan1
test_ln['banchan2'] = banchan2
test_ln['banchan3'] = banchan3
test_ln['kimchi'] = kimchi 
test_ln['side'] = side
# test_ln.iloc[1066:1070, 7:]
test_ln.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식_조리_기타,중식메인조리법,석식메인조리법,bob,gook,banchan1,banchan2,banchan3,kimchi,side
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,...,0.0,4,1,쌀밥/흑미밥/찰현미밥,대구지리,매운돈갈비찜,오꼬노미계란말이,상추무침,양상추샐러드*딸기D,포기김치
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,...,0.0,2,5,쌀밥/보리밥/찰현미밥,우렁된장찌개,오리주물럭,청양부추전,수제삼색무쌈,양상추샐러드*오미자D,겉절이김치
2,2021-01-29,5,2983,270,249,0,294,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,...,0.0,7,4,쌀밥/흑미밥/찰현미밥,팽이장국,수제돈까스*소스,가자미조림,동초나물무침,양상추샐러드*파인요거트D,포기김치
3,2021-02-01,1,2924,108,154,538,322,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,...,0.0,2,6,쌀밥/흑미밥/찰현미밥,배추들깨국,오리대패불고기,시금치프리타타,부추고추장무침,양상추샐러드*망고D,포기김치
4,2021-02-02,2,2924,62,186,455,314,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,...,0.0,6,1,쌀밥/팥밥/찰현미밥,부대찌개,닭살데리야끼조림,버섯탕수,세발나물무침,양상추샐러드*오리엔탈D,알타리김치/사과푸딩


In [426]:
# train_ln = train_ln[['year','month','day','dow','employees', 'dayoff', 'bustrip', 'ovtime', 'remote','real_emp','bob','gook','banchan1','banchan2','banchan3', 'kimchi','side','target_ln']]
# test_ln = test_ln[['year','month','day','dow','employees', 'dayoff', 'bustrip', 'ovtime', 'remote','real_emp','bob','gook','banchan1','banchan2','banchan3', 'kimchi','side']]

In [427]:
test_ln

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식_조리_기타,중식메인조리법,석식메인조리법,bob,gook,banchan1,banchan2,banchan3,kimchi,side
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,...,0.0,4,1,쌀밥/흑미밥/찰현미밥,대구지리,매운돈갈비찜,오꼬노미계란말이,상추무침,양상추샐러드*딸기D,포기김치
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,...,0.0,2,5,쌀밥/보리밥/찰현미밥,우렁된장찌개,오리주물럭,청양부추전,수제삼색무쌈,양상추샐러드*오미자D,겉절이김치
2,2021-01-29,5,2983,270,249,0,294,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,...,0.0,7,4,쌀밥/흑미밥/찰현미밥,팽이장국,수제돈까스*소스,가자미조림,동초나물무침,양상추샐러드*파인요거트D,포기김치
3,2021-02-01,1,2924,108,154,538,322,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,...,0.0,2,6,쌀밥/흑미밥/찰현미밥,배추들깨국,오리대패불고기,시금치프리타타,부추고추장무침,양상추샐러드*망고D,포기김치
4,2021-02-02,2,2924,62,186,455,314,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,...,0.0,6,1,쌀밥/팥밥/찰현미밥,부대찌개,닭살데리야끼조림,버섯탕수,세발나물무침,양상추샐러드*오리엔탈D,알타리김치/사과푸딩
5,2021-02-03,3,2924,59,199,5,286,모닝롤/게살모닝샌드 우유/주스 계란후라이/찐계란 소고기죽/흑미밥 시래기된장국 베이컨...,쌀밥/흑미밥/찰현미밥 아욱국 매콤해물볶음 감자조림 미나리나물 포기김치 콥샐러드*렌치D,오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치,...,0.0,1,2,쌀밥/흑미밥/찰현미밥,아욱국,매콤해물볶음,감자조림,미나리나물,콥샐러드*렌치D,포기김치
6,2021-02-04,4,2924,61,211,476,288,모닝롤/사과파이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 아욱국 새송이버섯곤약장...,쌀밥/차조밥/찰현미밥 설렁탕 고등어김치말이찜 볼어묵굴소스볶음 브로콜리숙회*초장 석박...,흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치,...,0.0,4,1,쌀밥/차조밥/찰현미밥,설렁탕,고등어김치말이찜,볼어묵굴소스볶음,브로콜리숙회*초장,양상추샐러드*키위D,석박지
7,2021-02-05,5,2924,169,252,0,256,모닝롤/앙버터모닝빵 우유/주스 계란후라이/찐계란 고구마죽/흑미밥 옹심이국 머위나물무...,쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러...,유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치,...,1.0,1,9,쌀밥/흑미밥/찰현미밥,북엇국,닭볶음탕,채소전*장,솎음열무나물무침,양상추샐러드*황도D,포기김치
8,2021-02-08,1,2924,88,174,690,329,모닝롤/스콘 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 꽃게탕 근대나물무침 연두부...,쌀밥/흑미밥/찰현미밥 감자양파국 돈수육*씨앗쌈장 매콤어묵볶음 콩나물파채무침 포기김치...,흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치,...,0.0,4,7,쌀밥/흑미밥/찰현미밥,감자양파국,돈수육*씨앗쌈장,매콤어묵볶음,콩나물파채무침,양상추샐러드*자몽D,포기김치
9,2021-02-09,2,2924,94,183,542,329,모닝롤/치즈팡샌드 우유/주스 계란후라이/찐계란 팥죽/흑미밥 맑은버섯국 시금치나물무침...,쌀밥/기장밥/찰현미밥 장각백숙 적어양념장구이 채소스틱*쌈장 도라지오이초무침 겉절이김...,흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지,...,0.0,2,2,쌀밥/기장밥/찰현미밥,장각백숙,적어양념장구이,채소스틱*쌈장,도라지오이초무침,양상추샐러드*참깨D,겉절이김치


In [428]:
# train_dn
dinner = []
for day in range(len(train)):
    tmp = train.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner.append(tmp)

In [429]:
dinner

[['쌀밥/잡곡밥', '육개장', '자반고등어구이', '두부조림', '건파래무침', '포기김치'],
 ['콩나물밥*양념장', '어묵국', '유산슬', '아삭고추무침', '바나나', '포기김치'],
 ['쌀밥/잡곡밥', '청국장찌개', '황태양념구이', '고기전', '새송이버섯볶음', '포기김치'],
 ['미니김밥*겨자장', '우동', '멕시칸샐러드', '군고구마', '무피클', '포기김치'],
 ['쌀밥/잡곡밥', '차돌박이찌개', '닭갈비', '감자소세지볶음', '콩나물무침', '포기김치'],
 ['참치회덮밥', '맑은국', '군만두', '과일샐러드', '락교', '포기김치'],
 ['쌀밥/잡곡밥', '김치콩나물국', '미니함박', '어묵볶음', '물파래무침', '깍두기'],
 ['쌀밥/잡곡밥', '홍합미역국', '등갈비김치찜', '임연수구이', '브로컬리초장', '포기김치'],
 ['쌀밥/잡곡밥', '된장찌개', '쇠불고기', '해파리겨자채', '봄동무침', '포기김치'],
 ['볶음밥*자장소스', '맑은국', '새우또띠아', '쨔샤이무침', '요플레', '포기김치'],
 ['쌀밥/잡곡밥', '순두부찌개', '명태조림', '감자채볶음', '쌈만두', '포기김치'],
 ['쌀밥/잡곡밥', '설렁탕', '김치전', '야채스틱', '바나나', '석박지'],
 ['쌀밥/잡곡밥', '옹심이만두국', '떡갈비조림', '청포묵무침', '부추오이생채', '포기김치'],
 ['쌀밥/잡곡밥', '콩비지찌개', '해물까스&타르타르소스', '볶음쌀국수', '미역레몬초무침', '통배추겉절이'],
 ['쌀밥/잡곡밥', '열무된장국', '매운돼지갈비찜', '멸치호두볶음', '숙주나물', '포기김치'],
 ['김밥', '잔치국수', '교자만두', '찜)', '멕시칸샐러드', '무피클', '포기김치'],
 ['쌀밥/잡곡밥', '닭개장', '삼치구이', '꽃맛살볶음', '청경채사과무침', '포기김치'],
 ['쌀밥/잡곡밥', '버섯들깨탕', '꽁치한마리구이', '두부양념조림', '구이김'

In [430]:
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []
for i, day_menu in enumerate(dinner):
    if (len(day_menu) < 4 ):
        bob.append('*')
        gook.append('*')
        banchan1.append('*')
        banchan2.append('*')
        banchan3.append('*')

        if i < 1067:
            kimchi.append('*')
            side.append('*')
        else: 
            kimchi.append('*')
            side.append('*')

    elif (len(day_menu)==4):
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3.append('*')

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    
    else :
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    

train_dn = train.copy()

train_dn['bob'] = bob
train_dn['gook'] = gook
train_dn['banchan1'] = banchan1
train_dn['banchan2'] = banchan2
train_dn['banchan3'] = banchan3
train_dn['kimchi'] = kimchi
train_dn['side'] = side
train_dn.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식_조리_기타,중식메인조리법,석식메인조리법,bob,gook,banchan1,banchan2,banchan3,kimchi,side
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",...,1.0,2,9,쌀밥/잡곡밥,육개장,자반고등어구이,두부조림,건파래무침,포기김치,건파래무침
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",...,1.0,7,9,콩나물밥*양념장,어묵국,유산슬,아삭고추무침,바나나,포기김치,바나나
2,2016-02-03,3,2601,56,180,111,0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",...,1.0,7,9,쌀밥/잡곡밥,청국장찌개,황태양념구이,고기전,새송이버섯볶음,포기김치,새송이버섯볶음
3,2016-02-04,4,2601,104,220,355,0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",...,1.0,1,9,미니김밥*겨자장,우동,멕시칸샐러드,군고구마,무피클,포기김치,무피클
4,2016-02-05,5,2601,278,181,34,0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",...,1.0,7,9,쌀밥/잡곡밥,차돌박이찌개,닭갈비,감자소세지볶음,콩나물무침,포기김치,콩나물무침


In [431]:
# test_dn
dinner = []
for day in range(len(test)):
    tmp = test.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner.append(tmp)

In [432]:
train.iloc[1, 9]

'콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국  유산슬 (쇠고기:호주산) 아삭고추무침  바나나  포기김치 (배추,고추가루:국내산) '

In [433]:
dinner

[['흑미밥', '얼큰순두부찌개', '쇠고기우엉볶음', '버섯햄볶음', '포기김치'],
 ['충무김밥', '우동국물', '오징어무침', '꽃맛살샐러드', '얼갈이쌈장무침', '석박지'],
 ['흑미밥', '물만둣국', '카레찜닭', '숯불양념꼬지어묵', '꼬시래기무침', '포기김치'],
 ['흑미밥', '동태탕', '돈육꽈리고추장조림', '당면채소무침', '모자반무침', '포기김치'],
 ['흑미밥', '바지락살국', '쇠고기청경채볶음', '두부구이*볶은김치', '머위된장무침', '백김치'],
 ['오므라이스', '가쓰오장국', '빌소세지구이*구운채소', '단감치커리무침', '양념고추지', '겉절이김치'],
 ['흑미밥', '계란파국', '돈육두루치기', '감자채파프리카볶음', '세발나물오리엔탈무침', '포기김치'],
 ['유부초밥/추가밥', '온메밀소바', '국물떡볶이', '순대찜*소금', '청경채겉절이', '포기김치'],
 ['흑미밥', '냉이국', '반반치킨', '꼬막채소무침', '청경채찜', '포기김치'],
 ['흑미밥', '미역국', '매운소불고기', '단호박두부탕수', '메추리알장조림', '석박지'],
 ['흑미밥', '참치김치찌개', '오징어굴소스볶음', '차돌비빔국수', '건새우무나물', '포기김치'],
 ['흑미밥', '순두부백탕', '수제치킨까스', '쫄면채소무침', '얼갈이나물', '포기김치'],
 ['흑미밥', '손수제비국', '쇠고기낙지볶음', '카레홍합찜', '쑥갓나물', '포기김치'],
 ['곤드레밥', '황태국', '찰떡떡갈비조림', '계란후라이', '재래김*달래양념장', '무생채'],
 ['흑미밥', '바지락된장찌개', '제육볶음', '양배추숙*쌈장', '노가리고추조림', '겉절이김치'],
 ['흑미밥', '버섯들깨탕', '아귀콩나물찜', '콤비네이션피자', '돌나물&된장소스', '포기김치'],
 ['흑미밥', '동태알탕', '깐풍육', '고사리볶음', '오이무침', '포기김치'],
 ['흑미밥', '쇠고기무국', '춘전닭갈비',

In [434]:
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []

for i, day_menu in enumerate(dinner):
    if (len(day_menu) < 4 ):
        bob.append('*')
        gook.append('*')
        banchan1.append('*')
        banchan2.append('*')
        banchan3.append('*')

        if i < 1067:
            kimchi.append('*')
            side.append('*')
        else: 
            kimchi.append('*')
            side.append('*')

    elif (len(day_menu)==4):
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3.append('*')

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    
    else :
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    

test_dn = test.copy()

test_dn['bob'] = bob
test_dn['gook'] = gook
test_dn['banchan1'] = banchan1
test_dn['banchan2'] = banchan2
test_dn['banchan3'] = banchan3
test_dn['kimchi'] = kimchi
test_dn['side'] = side
test_dn.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식_조리_기타,중식메인조리법,석식메인조리법,bob,gook,banchan1,banchan2,banchan3,kimchi,side
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,...,0.0,4,1,흑미밥,얼큰순두부찌개,쇠고기우엉볶음,버섯햄볶음,포기김치,포기김치,버섯햄볶음
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,...,0.0,2,5,충무김밥,우동국물,오징어무침,꽃맛살샐러드,얼갈이쌈장무침,석박지,얼갈이쌈장무침
2,2021-01-29,5,2983,270,249,0,294,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,...,0.0,7,4,흑미밥,물만둣국,카레찜닭,숯불양념꼬지어묵,꼬시래기무침,포기김치,꼬시래기무침
3,2021-02-01,1,2924,108,154,538,322,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,...,0.0,2,6,흑미밥,동태탕,돈육꽈리고추장조림,당면채소무침,모자반무침,포기김치,모자반무침
4,2021-02-02,2,2924,62,186,455,314,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,...,0.0,6,1,흑미밥,바지락살국,쇠고기청경채볶음,두부구이*볶은김치,머위된장무침,백김치,머위된장무침


In [435]:
# train_ln
원재료1 = [
    train_ln["banchan1"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    train_ln["banchan1"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    train_ln["banchan1"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    train_ln["banchan1"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    train_ln["banchan1"].str.contains('오리'),
    train_ln["banchan1"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist1 = ['해산물', '소', '닭',"돼지",'오리','채소']

train_ln['반찬1_원재료'] = np.select(원재료1, choicelist1, default='기타')

In [436]:
원재료2 = [
    train_ln["banchan2"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    train_ln["banchan2"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    train_ln["banchan2"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    train_ln["banchan2"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    train_ln["banchan2"].str.contains('오리'),
    train_ln["banchan2"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist2 = ['해산물', '소', '닭',"돼지",'오리','채소']

train_ln['반찬2_원재료'] = np.select(원재료2, choicelist2, default='기타')

In [437]:
원재료3 = [
    train_ln["banchan3"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    train_ln["banchan3"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    train_ln["banchan3"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    train_ln["banchan3"].str.contains('삽겹|폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    train_ln["banchan3"].str.contains('오리'),
    train_ln["banchan3"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist3 = ['해산물', '소', '닭',"돼지",'오리','채소']

train_ln['반찬3_원재료'] = np.select(원재료3, choicelist3, default='기타')

In [438]:
train_ln

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,bob,gook,banchan1,banchan2,banchan3,kimchi,side,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",...,쌀밥/잡곡밥,오징어찌개,쇠불고기,계란찜,청포묵무침,포기김치,요구르트,소,기타,기타
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",...,쌀밥/잡곡밥,김치찌개,가자미튀김,모둠소세지구이,마늘쫑무침,배추겉절이,요구르트,해산물,소,기타
2,2016-02-03,3,2601,56,180,111,0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",...,카레덮밥,팽이장국,치킨핑거,쫄면야채무침,견과류조림,포기김치,요구르트,닭,채소,기타
3,2016-02-04,4,2601,104,220,355,0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",...,쌀밥/잡곡밥,쇠고기무국,주꾸미볶음,부추전,시금치나물,포기김치,요구르트,해산물,채소,기타
4,2016-02-05,5,2601,278,181,34,0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",...,쌀밥/잡곡밥,떡국,돈육씨앗강정,우엉잡채,청경채무침,포기김치,요구르트,돼지,기타,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,3,2983,75,198,4,391,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,...,쌀밥/흑미밥/찰현미밥,아욱국,수제함박스테이크,견과류마카로니범벅,생깻잎지,단호박물김치,양상추샐러드*오리엔탈D,소,기타,채소
1201,2021-01-21,4,2983,92,231,462,351,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,...,쌀밥/수수밥/찰현미밥,냉이된장국,동파육,봄동전,청경채/버섯숙회*초장,무생채,양상추샐러드*파인D,돼지,기타,기타
1202,2021-01-22,5,2983,255,248,1,303,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,...,전주비빔밥*약고추장,계란파국,요거닭,올방개묵무침,파프리카해초무침,포기김치,양상추샐러드*자몽D,닭,기타,기타
1203,2021-01-25,1,2983,107,153,616,327,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,...,쌀밥/흑미밥/찰현미밥,전주식콩나물해장국,돈육간장불고기,깐풍연근,연두부*달래양념장,봄동겉절이/양념김,양상추샐러드*오렌지D,소,닭,채소


In [439]:
# test_ln
원재료1 = [
    test_ln["banchan1"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    test_ln["banchan1"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    test_ln["banchan1"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    test_ln["banchan1"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    test_ln["banchan1"].str.contains('오리'),
    test_ln["banchan1"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist1 = ['해산물', '소', '닭',"돼지",'오리','채소']

test_ln['반찬1_원재료'] = np.select(원재료1, choicelist1, default='기타')

In [440]:
원재료2 = [
    test_ln["banchan2"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    test_ln["banchan2"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    test_ln["banchan2"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    test_ln["banchan2"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    test_ln["banchan2"].str.contains('오리'),
    test_ln["banchan2"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist2 = ['해산물', '소', '닭',"돼지",'오리','채소']

test_ln['반찬2_원재료'] = np.select(원재료2, choicelist2, default='기타')

In [441]:
원재료3 = [
    test_ln["banchan3"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    test_ln["banchan3"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    test_ln["banchan3"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    test_ln["banchan3"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    test_ln["banchan3"].str.contains('오리'),
    test_ln["banchan3"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist3 = ['해산물', '소', '닭',"돼지",'오리','채소']

test_ln['반찬3_원재료'] = np.select(원재료3, choicelist3, default='기타')

In [442]:
test_ln

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,bob,gook,banchan1,banchan2,banchan3,kimchi,side,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,...,쌀밥/흑미밥/찰현미밥,대구지리,매운돈갈비찜,오꼬노미계란말이,상추무침,양상추샐러드*딸기D,포기김치,소,기타,기타
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,...,쌀밥/보리밥/찰현미밥,우렁된장찌개,오리주물럭,청양부추전,수제삼색무쌈,양상추샐러드*오미자D,겉절이김치,오리,채소,기타
2,2021-01-29,5,2983,270,249,0,294,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,...,쌀밥/흑미밥/찰현미밥,팽이장국,수제돈까스*소스,가자미조림,동초나물무침,양상추샐러드*파인요거트D,포기김치,소,해산물,기타
3,2021-02-01,1,2924,108,154,538,322,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,...,쌀밥/흑미밥/찰현미밥,배추들깨국,오리대패불고기,시금치프리타타,부추고추장무침,양상추샐러드*망고D,포기김치,소,기타,채소
4,2021-02-02,2,2924,62,186,455,314,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,...,쌀밥/팥밥/찰현미밥,부대찌개,닭살데리야끼조림,버섯탕수,세발나물무침,양상추샐러드*오리엔탈D,알타리김치/사과푸딩,닭,기타,기타
5,2021-02-03,3,2924,59,199,5,286,모닝롤/게살모닝샌드 우유/주스 계란후라이/찐계란 소고기죽/흑미밥 시래기된장국 베이컨...,쌀밥/흑미밥/찰현미밥 아욱국 매콤해물볶음 감자조림 미나리나물 포기김치 콥샐러드*렌치D,오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치,...,쌀밥/흑미밥/찰현미밥,아욱국,매콤해물볶음,감자조림,미나리나물,콥샐러드*렌치D,포기김치,해산물,채소,기타
6,2021-02-04,4,2924,61,211,476,288,모닝롤/사과파이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 아욱국 새송이버섯곤약장...,쌀밥/차조밥/찰현미밥 설렁탕 고등어김치말이찜 볼어묵굴소스볶음 브로콜리숙회*초장 석박...,흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치,...,쌀밥/차조밥/찰현미밥,설렁탕,고등어김치말이찜,볼어묵굴소스볶음,브로콜리숙회*초장,양상추샐러드*키위D,석박지,해산물,소,기타
7,2021-02-05,5,2924,169,252,0,256,모닝롤/앙버터모닝빵 우유/주스 계란후라이/찐계란 고구마죽/흑미밥 옹심이국 머위나물무...,쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러...,유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치,...,쌀밥/흑미밥/찰현미밥,북엇국,닭볶음탕,채소전*장,솎음열무나물무침,양상추샐러드*황도D,포기김치,닭,소,기타
8,2021-02-08,1,2924,88,174,690,329,모닝롤/스콘 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 꽃게탕 근대나물무침 연두부...,쌀밥/흑미밥/찰현미밥 감자양파국 돈수육*씨앗쌈장 매콤어묵볶음 콩나물파채무침 포기김치...,흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치,...,쌀밥/흑미밥/찰현미밥,감자양파국,돈수육*씨앗쌈장,매콤어묵볶음,콩나물파채무침,양상추샐러드*자몽D,포기김치,돼지,기타,기타
9,2021-02-09,2,2924,94,183,542,329,모닝롤/치즈팡샌드 우유/주스 계란후라이/찐계란 팥죽/흑미밥 맑은버섯국 시금치나물무침...,쌀밥/기장밥/찰현미밥 장각백숙 적어양념장구이 채소스틱*쌈장 도라지오이초무침 겉절이김...,흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지,...,쌀밥/기장밥/찰현미밥,장각백숙,적어양념장구이,채소스틱*쌈장,도라지오이초무침,양상추샐러드*참깨D,겉절이김치,해산물,소,기타


In [443]:
# train_dn
원재료1 = [
    train_dn["banchan1"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    train_dn["banchan1"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    train_dn["banchan1"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    train_dn["banchan1"].str.contains('삽겹|폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    train_dn["banchan1"].str.contains('오리'),
    train_dn["banchan1"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist1 = ['해산물', '소', '닭',"돼지",'오리','채소']

train_dn['반찬1_원재료'] = np.select(원재료1, choicelist1, default='기타')

In [444]:
원재료2 = [
    train_dn["banchan2"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    train_dn["banchan2"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    train_dn["banchan2"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    train_dn["banchan2"].str.contains('삽겹|폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    train_dn["banchan2"].str.contains('오리'),
    train_dn["banchan2"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist2 = ['해산물', '소', '닭',"돼지",'오리','채소']

train_dn['반찬2_원재료'] = np.select(원재료2, choicelist2, default='기타')

In [445]:
원재료3 = [
    train_dn["banchan3"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    train_dn["banchan3"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    train_dn["banchan3"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    train_dn["banchan3"].str.contains('삽겹|폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    train_dn["banchan3"].str.contains('오리'),
    train_dn["banchan3"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist3 = ['해산물', '소', '닭',"돼지",'오리','채소']

train_dn['반찬3_원재료'] = np.select(원재료3, choicelist3, default='기타')

In [446]:
train_dn

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,bob,gook,banchan1,banchan2,banchan3,kimchi,side,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,2016-02-01,1,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",...,쌀밥/잡곡밥,육개장,자반고등어구이,두부조림,건파래무침,포기김치,건파래무침,해산물,채소,기타
1,2016-02-02,2,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",...,콩나물밥*양념장,어묵국,유산슬,아삭고추무침,바나나,포기김치,바나나,해산물,채소,기타
2,2016-02-03,3,2601,56,180,111,0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",...,쌀밥/잡곡밥,청국장찌개,황태양념구이,고기전,새송이버섯볶음,포기김치,새송이버섯볶음,해산물,기타,기타
3,2016-02-04,4,2601,104,220,355,0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",...,미니김밥*겨자장,우동,멕시칸샐러드,군고구마,무피클,포기김치,무피클,기타,채소,기타
4,2016-02-05,5,2601,278,181,34,0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",...,쌀밥/잡곡밥,차돌박이찌개,닭갈비,감자소세지볶음,콩나물무침,포기김치,콩나물무침,소,소,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,3,2983,75,198,4,391,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,...,김치볶음밥,미니쫄우동*맛살튀김,브로콜리깨소스무침,계란후라이,고들빼기무침,고들빼기무침,겉절이김치,소,기타,기타
1201,2021-01-21,4,2983,92,231,462,351,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,...,흑미밥,쇠고기무국,삼치양념구이,비엔나채소볶음,숙주나물당근무침,숙주나물당근무침,포기김치,해산물,소,기타
1202,2021-01-22,5,2983,255,248,1,303,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,...,흑미밥,수제비국,수제맛쵸킹탕수육,유부채소겨자냉채,참나물무침,참나물무침,갓김치/겉절이김치,돼지,소,기타
1203,2021-01-25,1,2983,107,153,616,327,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,...,흑미밥,열무된장국,장어강정*데리야끼소스,깻잎쌈*생강채,오이선,오이선,포기김치,해산물,채소,기타


In [447]:
# test_dn
원재료1 = [
    test_dn["banchan1"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    test_dn["banchan1"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    test_dn["banchan1"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    test_dn["banchan1"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    test_dn["banchan1"].str.contains('오리'),
    test_dn["banchan1"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist1 = ['해산물', '소', '닭',"돼지",'오리','채소']

test_dn['반찬1_원재료'] = np.select(원재료1, choicelist1, default='기타')

In [448]:
원재료2 = [
    test_dn["banchan2"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    test_dn["banchan2"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    test_dn["banchan2"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    test_dn["banchan2"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    test_dn["banchan2"].str.contains('오리'),
    test_dn["banchan2"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist2 = ['해산물', '소', '닭',"돼지",'오리','채소']

test_dn['반찬2_원재료'] = np.select(원재료2, choicelist2, default='기타')

In [449]:
원재료3 = [
    test_dn["banchan3"].str.contains("황태|연어|맛살|골뱅이|열기|조기|탕수어|양장피|홍어|명태|적어|장어|동태|산슬|코다리|가자미|해물|생선|새우|꽁치|갈치|임연수|삼치|고등어|굴비|오징어|쭈꾸미|주꾸미|낙지|문어"),
    test_dn["banchan3"].str.contains('왕갈비|소갈비|장조림|불고기|차돌|육전|너비아니|떡갈비|소|소세|함박|쇠고기|소고기|쇠|갈비'),
    test_dn["banchan3"].str.contains('궁보계정|삼계탕|윙|유린기|깐풍|닭|치킨|후라이드|장각'),
    test_dn["banchan3"].str.contains('폭립|오향장육|동파육|히레카츠|순대|미트볼|등갈비|소세지|목살|탕수육|제육|돈|돼지|두루치기|삼겹|보쌈|족발'),
    test_dn["banchan3"].str.contains('오리'),
    test_dn["banchan3"].str.contains('두부|꼬치산적|고추|양파|부추|고구마|감자|깻잎|샐러드시금치|야채'),    
]

choicelist3 = ['해산물', '소', '닭',"돼지",'오리','채소']

test_dn['반찬3_원재료'] = np.select(원재료3, choicelist3, default='기타')

In [450]:
test_dn

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,bob,gook,banchan1,banchan2,banchan3,kimchi,side,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,2021-01-27,3,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,...,흑미밥,얼큰순두부찌개,쇠고기우엉볶음,버섯햄볶음,포기김치,포기김치,버섯햄볶음,소,기타,기타
1,2021-01-28,4,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,...,충무김밥,우동국물,오징어무침,꽃맛살샐러드,얼갈이쌈장무침,석박지,얼갈이쌈장무침,해산물,해산물,기타
2,2021-01-29,5,2983,270,249,0,294,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,...,흑미밥,물만둣국,카레찜닭,숯불양념꼬지어묵,꼬시래기무침,포기김치,꼬시래기무침,닭,기타,기타
3,2021-02-01,1,2924,108,154,538,322,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,...,흑미밥,동태탕,돈육꽈리고추장조림,당면채소무침,모자반무침,포기김치,모자반무침,소,소,기타
4,2021-02-02,2,2924,62,186,455,314,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,...,흑미밥,바지락살국,쇠고기청경채볶음,두부구이*볶은김치,머위된장무침,백김치,머위된장무침,소,채소,기타
5,2021-02-03,3,2924,59,199,5,286,모닝롤/게살모닝샌드 우유/주스 계란후라이/찐계란 소고기죽/흑미밥 시래기된장국 베이컨...,쌀밥/흑미밥/찰현미밥 아욱국 매콤해물볶음 감자조림 미나리나물 포기김치 콥샐러드*렌치D,오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치,...,오므라이스,가쓰오장국,빌소세지구이*구운채소,단감치커리무침,양념고추지,겉절이김치,양념고추지,소,기타,채소
6,2021-02-04,4,2924,61,211,476,288,모닝롤/사과파이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 아욱국 새송이버섯곤약장...,쌀밥/차조밥/찰현미밥 설렁탕 고등어김치말이찜 볼어묵굴소스볶음 브로콜리숙회*초장 석박...,흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치,...,흑미밥,계란파국,돈육두루치기,감자채파프리카볶음,세발나물오리엔탈무침,포기김치,세발나물오리엔탈무침,돼지,채소,오리
7,2021-02-05,5,2924,169,252,0,256,모닝롤/앙버터모닝빵 우유/주스 계란후라이/찐계란 고구마죽/흑미밥 옹심이국 머위나물무...,쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러...,유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치,...,유부초밥/추가밥,온메밀소바,국물떡볶이,순대찜*소금,청경채겉절이,포기김치,청경채겉절이,기타,소,기타
8,2021-02-08,1,2924,88,174,690,329,모닝롤/스콘 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 꽃게탕 근대나물무침 연두부...,쌀밥/흑미밥/찰현미밥 감자양파국 돈수육*씨앗쌈장 매콤어묵볶음 콩나물파채무침 포기김치...,흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치,...,흑미밥,냉이국,반반치킨,꼬막채소무침,청경채찜,포기김치,청경채찜,닭,소,기타
9,2021-02-09,2,2924,94,183,542,329,모닝롤/치즈팡샌드 우유/주스 계란후라이/찐계란 팥죽/흑미밥 맑은버섯국 시금치나물무침...,쌀밥/기장밥/찰현미밥 장각백숙 적어양념장구이 채소스틱*쌈장 도라지오이초무침 겉절이김...,흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지,...,흑미밥,미역국,매운소불고기,단호박두부탕수,메추리알장조림,석박지,메추리알장조림,소,채소,소


In [451]:
train_ln['반찬1_원재료'].value_counts(), train_ln['반찬2_원재료'].value_counts(),train_ln['반찬3_원재료'].value_counts()

(소      402
 해산물    300
 돼지     206
 닭      187
 오리      42
 채소      39
 기타      29
 Name: 반찬1_원재료, dtype: int64,
 기타     647
 채소     252
 해산물    228
 소       54
 닭       14
 돼지      10
 Name: 반찬2_원재료, dtype: int64,
 기타     958
 채소     205
 해산물     20
 소       14
 닭        7
 돼지       1
 Name: 반찬3_원재료, dtype: int64)

In [452]:
train_dn['반찬1_원재료'].value_counts(), train_dn['반찬2_원재료'].value_counts(),train_dn['반찬3_원재료'].value_counts()

(해산물    337
 소      252
 기타     236
 돼지     159
 닭      138
 채소      69
 오리      14
 Name: 반찬1_원재료, dtype: int64,
 기타     692
 채소     251
 해산물    167
 소       58
 돼지      20
 닭       12
 오리       5
 Name: 반찬2_원재료, dtype: int64,
 기타     1044
 채소      123
 해산물      21
 소        11
 닭         3
 오리        2
 돼지        1
 Name: 반찬3_원재료, dtype: int64)

In [453]:
train_ln = train_ln[['bob', 'gook', 'banchan1', 'banchan2', 'banchan3', 'kimchi', 'side',
        '반찬1_원재료', '반찬2_원재료', '반찬3_원재료']].copy()
test_ln = test_ln[['bob',
        'gook', 'banchan1', 'banchan2', 'banchan3', 'kimchi', 'side', '반찬1_원재료',
        '반찬2_원재료', '반찬3_원재료']].copy()
train_dn = train_dn[['bob',
        'gook', 'banchan1', 'banchan2', 'banchan3', 'kimchi', 'side', '반찬1_원재료',
        '반찬2_원재료', '반찬3_원재료']].copy()
test_dn = test_dn[['bob',
        'gook', 'banchan1', 'banchan2', 'banchan3', 'kimchi', 'side', '반찬1_원재료',
        '반찬2_원재료', '반찬3_원재료']].copy()

In [454]:
train_ln.columns = ['밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료', '반찬2_원재료', '반찬3_원재료']
test_ln.columns = ['밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료', '반찬2_원재료', '반찬3_원재료']
train_dn.columns = ['밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료', '반찬2_원재료', '반찬3_원재료']
test_dn.columns = ['밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료', '반찬2_원재료', '반찬3_원재료']

In [455]:
train_ln.head()

,밥,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,쌀밥/잡곡밥,오징어찌개,쇠불고기,계란찜,청포묵무침,포기김치,요구르트,소,기타,기타
1,쌀밥/잡곡밥,김치찌개,가자미튀김,모둠소세지구이,마늘쫑무침,배추겉절이,요구르트,해산물,소,기타
2,카레덮밥,팽이장국,치킨핑거,쫄면야채무침,견과류조림,포기김치,요구르트,닭,채소,기타
3,쌀밥/잡곡밥,쇠고기무국,주꾸미볶음,부추전,시금치나물,포기김치,요구르트,해산물,채소,기타
4,쌀밥/잡곡밥,떡국,돈육씨앗강정,우엉잡채,청경채무침,포기김치,요구르트,돼지,기타,기타


In [456]:
test_ln.head()

,밥,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,쌀밥/흑미밥/찰현미밥,대구지리,매운돈갈비찜,오꼬노미계란말이,상추무침,양상추샐러드*딸기D,포기김치,소,기타,기타
1,쌀밥/보리밥/찰현미밥,우렁된장찌개,오리주물럭,청양부추전,수제삼색무쌈,양상추샐러드*오미자D,겉절이김치,오리,채소,기타
2,쌀밥/흑미밥/찰현미밥,팽이장국,수제돈까스*소스,가자미조림,동초나물무침,양상추샐러드*파인요거트D,포기김치,소,해산물,기타
3,쌀밥/흑미밥/찰현미밥,배추들깨국,오리대패불고기,시금치프리타타,부추고추장무침,양상추샐러드*망고D,포기김치,소,기타,채소
4,쌀밥/팥밥/찰현미밥,부대찌개,닭살데리야끼조림,버섯탕수,세발나물무침,양상추샐러드*오리엔탈D,알타리김치/사과푸딩,닭,기타,기타


In [457]:
train_dn.head()

,밥,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,쌀밥/잡곡밥,육개장,자반고등어구이,두부조림,건파래무침,포기김치,건파래무침,해산물,채소,기타
1,콩나물밥*양념장,어묵국,유산슬,아삭고추무침,바나나,포기김치,바나나,해산물,채소,기타
2,쌀밥/잡곡밥,청국장찌개,황태양념구이,고기전,새송이버섯볶음,포기김치,새송이버섯볶음,해산물,기타,기타
3,미니김밥*겨자장,우동,멕시칸샐러드,군고구마,무피클,포기김치,무피클,기타,채소,기타
4,쌀밥/잡곡밥,차돌박이찌개,닭갈비,감자소세지볶음,콩나물무침,포기김치,콩나물무침,소,소,기타


In [458]:
test_dn.head()

,밥,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,흑미밥,얼큰순두부찌개,쇠고기우엉볶음,버섯햄볶음,포기김치,포기김치,버섯햄볶음,소,기타,기타
1,충무김밥,우동국물,오징어무침,꽃맛살샐러드,얼갈이쌈장무침,석박지,얼갈이쌈장무침,해산물,해산물,기타
2,흑미밥,물만둣국,카레찜닭,숯불양념꼬지어묵,꼬시래기무침,포기김치,꼬시래기무침,닭,기타,기타
3,흑미밥,동태탕,돈육꽈리고추장조림,당면채소무침,모자반무침,포기김치,모자반무침,소,소,기타
4,흑미밥,바지락살국,쇠고기청경채볶음,두부구이*볶은김치,머위된장무침,백김치,머위된장무침,소,채소,기타


In [459]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '연도', '월', '일',
       '출근인원', '중식_신메뉴', '석식_신메뉴', '휴일전날', '중식_전', '중식_무침', '중식_튀김', '중식_찜',
       '중식_볶음', '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타', '석식_전', '석식_무침',
       '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타',
       '중식메인조리법', '석식메인조리법'],
      dtype='object')

In [460]:
train_ln.columns

Index(['밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료', '반찬2_원재료',
       '반찬3_원재료'],
      dtype='object')

In [461]:
train_lunch = pd.concat([train[['연도', '월', '일', '요일', '출근인원', '중식_신메뉴', '휴일전날', '중식_전', '중식_무침', '중식_튀김', '중식_찜', '중식_볶음',
       '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타', '중식메인조리법']],
                         train_ln, train[['중식계']]], axis=1)

train_lunch.head()

,연도,월,일,요일,출근인원,중식_신메뉴,휴일전날,중식_전,중식_무침,중식_튀김,...,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료,중식계
0,2016,2,1,1,2401,0,0.0,0.0,0.0,0.0,...,오징어찌개,쇠불고기,계란찜,청포묵무침,포기김치,요구르트,소,기타,기타,1039
1,2016,2,2,2,2378,0,0.0,0.0,0.0,1.0,...,김치찌개,가자미튀김,모둠소세지구이,마늘쫑무침,배추겉절이,요구르트,해산물,소,기타,867
2,2016,2,3,3,2365,0,0.0,0.0,0.0,1.0,...,팽이장국,치킨핑거,쫄면야채무침,견과류조림,포기김치,요구르트,닭,채소,기타,1017
3,2016,2,4,4,2277,0,0.0,0.0,0.0,0.0,...,쇠고기무국,주꾸미볶음,부추전,시금치나물,포기김치,요구르트,해산물,채소,기타,978
4,2016,2,5,5,2142,0,1.0,0.0,0.0,1.0,...,떡국,돈육씨앗강정,우엉잡채,청경채무침,포기김치,요구르트,돼지,기타,기타,925


In [462]:
test_lunch = pd.concat([test[['연도', '월', '일', '요일', '출근인원', '중식_신메뉴', '휴일전날', '중식_전', '중식_무침', '중식_튀김', '중식_찜', '중식_볶음',
       '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타', '중식메인조리법']],
                         test_ln], axis=1)

test_lunch.head()

,연도,월,일,요일,출근인원,중식_신메뉴,휴일전날,중식_전,중식_무침,중식_튀김,...,밥,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,2021,1,27,3,2355,0,0.0,0.0,0.0,0.0,...,쌀밥/흑미밥/찰현미밥,대구지리,매운돈갈비찜,오꼬노미계란말이,상추무침,양상추샐러드*딸기D,포기김치,소,기타,기타
1,2021,1,28,4,2319,0,0.0,0.0,0.0,0.0,...,쌀밥/보리밥/찰현미밥,우렁된장찌개,오리주물럭,청양부추전,수제삼색무쌈,양상추샐러드*오미자D,겉절이김치,오리,채소,기타
2,2021,1,29,5,2170,0,1.0,0.0,0.0,1.0,...,쌀밥/흑미밥/찰현미밥,팽이장국,수제돈까스*소스,가자미조림,동초나물무침,양상추샐러드*파인요거트D,포기김치,소,해산물,기타
3,2021,2,1,1,2340,0,0.0,0.0,0.0,0.0,...,쌀밥/흑미밥/찰현미밥,배추들깨국,오리대패불고기,시금치프리타타,부추고추장무침,양상추샐러드*망고D,포기김치,소,기타,채소
4,2021,2,2,2,2362,0,0.0,0.0,0.0,0.0,...,쌀밥/팥밥/찰현미밥,부대찌개,닭살데리야끼조림,버섯탕수,세발나물무침,양상추샐러드*오리엔탈D,알타리김치/사과푸딩,닭,기타,기타


In [463]:
train_dinner = pd.concat([train[['연도', '월', '일', '요일', '출근인원', '석식_신메뉴', '휴일전날', '석식_전', '석식_무침',
       '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타', '석식메인조리법']],
                         train_dn, train[['석식계']]], axis=1)

train_dinner.head()

,연도,월,일,요일,출근인원,석식_신메뉴,휴일전날,석식_전,석식_무침,석식_튀김,...,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료,석식계
0,2016,2,1,1,2401,0,0.0,0.0,0.0,0.0,...,육개장,자반고등어구이,두부조림,건파래무침,포기김치,건파래무침,해산물,채소,기타,331
1,2016,2,2,2,2378,0,0.0,0.0,0.0,0.0,...,어묵국,유산슬,아삭고추무침,바나나,포기김치,바나나,해산물,채소,기타,560
2,2016,2,3,3,2365,0,0.0,0.0,0.0,0.0,...,청국장찌개,황태양념구이,고기전,새송이버섯볶음,포기김치,새송이버섯볶음,해산물,기타,기타,573
3,2016,2,4,4,2277,0,0.0,0.0,0.0,0.0,...,우동,멕시칸샐러드,군고구마,무피클,포기김치,무피클,기타,채소,기타,525
4,2016,2,5,5,2142,0,1.0,0.0,0.0,0.0,...,차돌박이찌개,닭갈비,감자소세지볶음,콩나물무침,포기김치,콩나물무침,소,소,기타,330


In [464]:
test_dinner = pd.concat([test[['연도', '월', '일', '요일', '출근인원', '석식_신메뉴', '휴일전날', '석식_전', '석식_무침',
       '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타', '석식메인조리법']],
                         test_dn], axis=1)

test_dinner.head()

,연도,월,일,요일,출근인원,석식_신메뉴,휴일전날,석식_전,석식_무침,석식_튀김,...,밥,국,반찬1,반찬2,반찬3,김치,사이드,반찬1_원재료,반찬2_원재료,반찬3_원재료
0,2021,1,27,3,2355,1,0.0,0.0,0.0,0.0,...,흑미밥,얼큰순두부찌개,쇠고기우엉볶음,버섯햄볶음,포기김치,포기김치,버섯햄볶음,소,기타,기타
1,2021,1,28,4,2319,0,0.0,0.0,1.0,0.0,...,충무김밥,우동국물,오징어무침,꽃맛살샐러드,얼갈이쌈장무침,석박지,얼갈이쌈장무침,해산물,해산물,기타
2,2021,1,29,5,2170,0,1.0,0.0,0.0,0.0,...,흑미밥,물만둣국,카레찜닭,숯불양념꼬지어묵,꼬시래기무침,포기김치,꼬시래기무침,닭,기타,기타
3,2021,2,1,1,2340,0,0.0,0.0,0.0,0.0,...,흑미밥,동태탕,돈육꽈리고추장조림,당면채소무침,모자반무침,포기김치,모자반무침,소,소,기타
4,2021,2,2,2,2362,0,0.0,0.0,0.0,0.0,...,흑미밥,바지락살국,쇠고기청경채볶음,두부구이*볶은김치,머위된장무침,백김치,머위된장무침,소,채소,기타


In [465]:
train_lunch.columns, test_lunch.columns, train_dinner.columns, test_dinner.columns

(Index(['연도', '월', '일', '요일', '출근인원', '중식_신메뉴', '휴일전날', '중식_전', '중식_무침',
        '중식_튀김', '중식_찜', '중식_볶음', '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타',
        '중식메인조리법', '밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료',
        '반찬2_원재료', '반찬3_원재료', '중식계'],
       dtype='object'),
 Index(['연도', '월', '일', '요일', '출근인원', '중식_신메뉴', '휴일전날', '중식_전', '중식_무침',
        '중식_튀김', '중식_찜', '중식_볶음', '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타',
        '중식메인조리법', '밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료',
        '반찬2_원재료', '반찬3_원재료'],
       dtype='object'),
 Index(['연도', '월', '일', '요일', '출근인원', '석식_신메뉴', '휴일전날', '석식_전', '석식_무침',
        '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타',
        '석식메인조리법', '밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드', '반찬1_원재료',
        '반찬2_원재료', '반찬3_원재료', '석식계'],
       dtype='object'),
 Index(['연도', '월', '일', '요일', '출근인원', '석식_신메뉴', '휴일전날', '석식_전', '석식_무침',
        '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타',
        '석식메인조리

In [ ]:
train = train[['연도', '월', '일', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수','출근인원','휴일전날', '조식메뉴', '중식메뉴', '석식메뉴', 
       '중식_신메뉴', '석식_신메뉴', '중식_전', '중식_무침', '중식_튀김', '중식_찜',
       '중식_볶음', '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타', '석식_전', '석식_무침',
       '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타',
       '중식메인조리법', '석식메인조리법', '중식계', '석식계']].copy()

In [ ]:
train = test[['연도', '월', '일', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수','출근인원','휴일전날', '조식메뉴', '중식메뉴', '석식메뉴', 
       '중식_신메뉴', '석식_신메뉴', '중식_전', '중식_무침', '중식_튀김', '중식_찜',
       '중식_볶음', '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타', '석식_전', '석식_무침',
       '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타',
       '중식메인조리법', '석식메인조리법']].copy()